In [1]:
!pip install scipy==1.7.0

In [2]:
!sudo apt-get install build-essential swig
!curl https://raw.githubusercontent.com/automl/auto-sklearn/master/requirements.txt | xargs -n 1 -L 1 pip install
!pip install auto-sklearn
import autosklearn.classification

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
swig is already the newest version (3.0.12-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   250  100   250    0     0   2100      0 --:--:-- --:--:-- --:--:--  2100


In [4]:
import sklearn.metrics
import pandas as pd
import numpy as np

In [5]:
# import the data set.
# do this directly from the github
url = "https://raw.githubusercontent.com/BUCKERS99/C7084-Big-data--Will-it-rain-tomorrow-/main/weather_data.csv"
df = pd.read_csv(url)
# print(c)

In [6]:
## EDA ####
 # showing the forst and last 5 rows of the data and all variables
df.head

<bound method NDFrame.head of               Date Location  MinTemp  MaxTemp  Rainfall  Evaporation  \
0       01/12/2008   Albury     13.4     22.9       0.6          NaN   
1       02/12/2008   Albury      7.4     25.1       0.0          NaN   
2       03/12/2008   Albury     12.9     25.7       0.0          NaN   
3       04/12/2008   Albury      9.2     28.0       0.0          NaN   
4       05/12/2008   Albury     17.5     32.3       1.0          NaN   
...            ...      ...      ...      ...       ...          ...   
145455  21/06/2017    Uluru      2.8     23.4       0.0          NaN   
145456  22/06/2017    Uluru      3.6     25.3       0.0          NaN   
145457  23/06/2017    Uluru      5.4     26.9       0.0          NaN   
145458  24/06/2017    Uluru      7.8     27.0       0.0          NaN   
145459  25/06/2017    Uluru     14.9      NaN       0.0          NaN   

        Sunshine WindGustDir  WindGustSpeed WindDir9am  ... Humidity9am  \
0            NaN           W  

In [7]:
# Sorting out data class
for col in ['WindDir9am', 'WindDir3pm', 'WindGustDir', 'RainToday', 'RainTomorrow']:
    df[col] = df[col].astype('category')
# looking at the data types and non-null values
df.info()
# this shows that there is some NA values

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype   
---  ------         --------------   -----   
 0   Date           145460 non-null  object  
 1   Location       145460 non-null  object  
 2   MinTemp        143975 non-null  float64 
 3   MaxTemp        144199 non-null  float64 
 4   Rainfall       142199 non-null  float64 
 5   Evaporation    82670 non-null   float64 
 6   Sunshine       75625 non-null   float64 
 7   WindGustDir    135134 non-null  category
 8   WindGustSpeed  135197 non-null  float64 
 9   WindDir9am     134894 non-null  category
 10  WindDir3pm     141232 non-null  category
 11  WindSpeed9am   143693 non-null  float64 
 12  WindSpeed3pm   142398 non-null  float64 
 13  Humidity9am    142806 non-null  float64 
 14  Humidity3pm    140953 non-null  float64 
 15  Pressure9am    130395 non-null  float64 
 16  Pressure3pm    130432 non-null  float64 
 17  Cloud9am  

In [8]:
# separate the model features and target
model_features = df.columns.drop(['RainTomorrow','Date', 'Location'])
#model_features = df.columns.drop('Date')
model_target = 'RainTomorrow'

print('Model features: ', model_features)
print('Model target: ', model_target)

Model features:  Index(['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine',
       'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday'],
      dtype='object')
Model target:  RainTomorrow


In [9]:
df.isna().sum()
# we can see that there are missing values in all of the varibles, but for the moment we will remove just the dependant variable NA's

Date                 0
Location             0
MinTemp           1485
MaxTemp           1261
Rainfall          3261
Evaporation      62790
Sunshine         69835
WindGustDir      10326
WindGustSpeed    10263
WindDir9am       10566
WindDir3pm        4228
WindSpeed9am      1767
WindSpeed3pm      3062
Humidity9am       2654
Humidity3pm       4507
Pressure9am      15065
Pressure3pm      15028
Cloud9am         55888
Cloud3pm         59358
Temp9am           1767
Temp3pm           3609
RainToday         3261
RainTomorrow      3267
dtype: int64

In [10]:
#df_NA = df.dropna(subset=["RainTomorrow"])
df_NA = df.dropna()
# run the code from line 53 again to check if this has removed the values
df_NA.isna().sum()

Date             0
Location         0
MinTemp          0
MaxTemp          0
Rainfall         0
Evaporation      0
Sunshine         0
WindGustDir      0
WindGustSpeed    0
WindDir9am       0
WindDir3pm       0
WindSpeed9am     0
WindSpeed3pm     0
Humidity9am      0
Humidity3pm      0
Pressure9am      0
Pressure3pm      0
Cloud9am         0
Cloud3pm         0
Temp9am          0
Temp3pm          0
RainToday        0
RainTomorrow     0
dtype: int64

In [ ]:
# we have to do some pre processing due to not being able to convert string to float (An error i have experienced many times now trying to run this)
#from sklearn.preprocessing import OrdinalEncoder
#ord_enc = OrdinalEncoder()
#df_NA["WindGustDir"] = ord_enc.fit_transform(df_NA[["WindGustDir"]])
#df_NA["WindDir9am"] = ord_enc.fit_transform(df_NA[["WindDir9am"]])
#df_NA["WindDir3pm"] = ord_enc.fit_transform(df_NA[["WindDir3pm"]])
#df_NA["RainTomorrow"] = ord_enc.fit_transform(df_NA[["RainTomorrow"]])
#df_NA["RainToday"] = ord_enc.fit_transform(df_NA[["RainToday"]])

In [11]:
# Our dependent variable is a factor of 2 levels (Categorical) so we will first run a KNN model to see what type of accuracy we achieve

# We first need to split the data set into train:test:validate 80:10:10
train_df, val_df, test_df = np.split(df_NA.sample(frac=1), [int(.80*len(df_NA)), int(.9*len(df_NA))])

In [12]:
# Check that they contain what we expect 113754:14219:14219
print(train_df.shape)
print(test_df.shape)
print(val_df.shape)
print("!!CORRECT!!")

(45136, 23)
(5642, 23)
(5642, 23)
!!CORRECT!!


In [13]:
x_train = train_df[model_features]
y_train = train_df[model_target]

x_test = test_df[model_features]
y_test = test_df[model_target]

x_val = val_df[model_features]
y_val = val_df[model_target]


print(model_features)

Index(['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine',
       'WindGustDir', 'WindGustSpeed', 'WindDir9am', 'WindDir3pm',
       'WindSpeed9am', 'WindSpeed3pm', 'Humidity9am', 'Humidity3pm',
       'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'Temp9am',
       'Temp3pm', 'RainToday'],
      dtype='object')


In [15]:
automl = autosklearn.classification.AutoSklearnClassifier()
automl.fit(x_train, y_train)

AutoSklearnClassifier(per_run_time_limit=360)

In [16]:
print(automl.leaderboard())

          rank  ensemble_weight                 type      cost    duration
model_id                                                                  
21           1             0.12          extra_trees  0.137899  129.405332
12           2             0.08    gradient_boosting  0.139040   17.205127
2            3             0.12        random_forest  0.139510   43.211077
6            4             0.08    gradient_boosting  0.140785  285.144307
16           5             0.06          extra_trees  0.143135   49.722666
25           6             0.22                  mlp  0.143337   60.898560
26           7             0.08          extra_trees  0.144209   33.277309
18           8             0.16    gradient_boosting  0.145351   25.674767
28           9             0.02          extra_trees  0.145821   22.319221
33          10             0.02   passive_aggressive  0.149648    5.183819
24          11             0.02                  lda  0.169050    5.722172
32          12           

In [27]:
print(automl.sprint_statistics())

auto-sklearn results:
  Dataset name: 0601fb3a-b10d-11ec-860c-0242ac1c0002
  Metric: accuracy
  Best validation score: 0.862101
  Number of target algorithm runs: 34
  Number of successful target algorithm runs: 28
  Number of crashed target algorithm runs: 1
  Number of target algorithms that exceeded the time limit: 5
  Number of target algorithms that exceeded the memory limit: 0



In [22]:
print(automl.show_models())

{21: {'model_id': 21, 'rank': 1, 'cost': 0.1378986236992279, 'ensemble_weight': 0.12, 'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7fbdeb5d4810>, 'balancing': Balancing(random_state=1), 'feature_preprocessor': <autosklearn.pipeline.components.feature_preprocessing.FeaturePreprocessorChoice object at 0x7fbdeb8a5350>, 'classifier': <autosklearn.pipeline.components.classification.ClassifierChoice object at 0x7fbdeb8d11d0>, 'sklearn_classifier': ExtraTreesClassifier(bootstrap=True, max_features=120, min_samples_leaf=2,
                     min_samples_split=9, n_estimators=512, n_jobs=1,
                     random_state=1, warm_start=True)}, 12: {'model_id': 12, 'rank': 2, 'cost': 0.1390399462907016, 'ensemble_weight': 0.08, 'data_preprocessor': <autosklearn.pipeline.components.data_preprocessing.DataPreprocessorChoice object at 0x7fbdeb5d4150>, 'balancing': Balancing(random_state=1), 'feature_preprocessor': <autosklearn.pipel

In [23]:
predictions = automl.predict(x_test)

In [25]:
print("Accuracy score", sklearn.metrics.accuracy_score(y_test, predictions))

Accuracy score 0.8642325416518964
